mAP Debugger/home/erikserrano/.vscode-server/extensions/ms-python.vscode-pylance-2024.12.1/dist/typeshed-fallback/stubs/tqdm/tqdm/contrib/concurrent.pyi

In [51]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from copairs import map


# these are the imports found within the average_percision.py module
import itertools
import logging

import numpy as np
import pandas as pd
from tqdm.contrib.concurrent import thread_map
from copairs import compute
from copairs.matching import Matcher, UnpairedException
from copairs.map.average_precision import build_rank_lists
from statsmodels.stats.multitest import multipletests

from copairs.map.filter import evaluate_and_filter, flatten_str_list, validate_pipeline_input

logger = logging.getLogger("copairs")

Downloading data

In [2]:
commit = "da8ae6a3bc103346095d61b4ee02f08fc85a5d98"
plate = "SQ00014812"
url = f"https://media.githubusercontent.com/media/broadinstitute/lincs-cell-painting/{commit}/profiles/2016_04_01_a549_48hr_batch1/{plate}/{plate}_normalized_feature_select.csv.gz"

df = pd.read_csv(url)
df = df.loc[:, df.nunique() > 1]  # remove constant columns
df

,Metadata_broad_sample,Metadata_mg_per_ml,Metadata_mmoles_per_liter,Metadata_pert_id,Metadata_pert_mfc_id,Metadata_pert_well,Metadata_broad_sample_type,Metadata_pert_type,Metadata_broad_id,Metadata_InChIKey14,...,Nuclei_Texture_InverseDifferenceMoment_AGP_5_0,Nuclei_Texture_InverseDifferenceMoment_DNA_20_0,Nuclei_Texture_InverseDifferenceMoment_ER_5_0,Nuclei_Texture_InverseDifferenceMoment_Mito_10_0,Nuclei_Texture_InverseDifferenceMoment_Mito_5_0,Nuclei_Texture_SumAverage_RNA_5_0,Nuclei_Texture_SumEntropy_DNA_10_0,Nuclei_Texture_SumEntropy_DNA_20_0,Nuclei_Texture_SumEntropy_DNA_5_0,Nuclei_Texture_Variance_RNA_10_0
0,DMSO,0.000000,0.000000,NaN,NaN,A01,control,control,NaN,NaN,...,-1.3544,-1.07770,2.26020,-0.377010,-0.065840,2.12360,2.8740,2.87500,2.3047,-0.92358
1,DMSO,0.000000,0.000000,NaN,NaN,A02,control,control,NaN,NaN,...,-2.3840,-0.73440,1.12090,-0.182500,-0.061450,0.66985,2.3919,2.35230,1.8672,-0.11820
2,DMSO,0.000000,0.000000,NaN,NaN,A03,control,control,NaN,NaN,...,-1.9493,-0.36148,0.44050,0.326660,0.547200,0.25015,1.2271,0.77847,1.0651,-0.44810
3,DMSO,0.000000,0.000000,NaN,NaN,A04,control,control,NaN,NaN,...,-2.2909,-0.46380,0.96434,1.132200,0.753500,0.31403,1.4384,1.48110,1.2943,-0.83810
4,DMSO,0.000000,0.000000,NaN,NaN,A05,control,control,NaN,NaN,...,-1.8955,-1.05350,1.64840,0.057781,0.070229,1.60990,1.1296,0.90213,1.1016,0.53225
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
379,BRD-K82746043-001-15-1,3.248700,3.333300,BRD-K82746043,BRD-K82746043-001-15-1,P20,trt,trt,BRD-K82746043,JLYAXFNOILIKPP,...,-6.1522,1.81410,1.54220,-1.874700,-1.133900,1.57540,-3.0962,-3.25160,-2.7683,1.40170
380,BRD-K82746043-001-15-1,1.082900,1.111100,BRD-K82746043,BRD-K82746043-001-15-1,P21,trt,trt,BRD-K82746043,JLYAXFNOILIKPP,...,-5.1586,1.50580,1.68420,-1.126400,-1.066600,1.24740,-1.5305,-1.79020,-1.2474,1.17600
381,BRD-K82746043-001-15-1,0.360970,0.370370,BRD-K82746043,BRD-K82746043-001-15-1,P22,trt,trt,BRD-K82746043,JLYAXFNOILIKPP,...,-5.9475,1.42100,1.51020,-1.103600,-1.666500,1.19840,-2.6086,-2.97620,-2.0026,0.91557
382,BRD-K82746043-001-15-1,0.120320,0.123460,BRD-K82746043,BRD-K82746043-001-15-1,P23,trt,trt,BRD-K82746043,JLYAXFNOILIKPP,...,-8.4408,2.99620,2.55230,-2.275200,-1.783500,2.49200,-4.3964,-4.19030,-3.8360,1.02240


In [3]:
# Metadata_pert_type = indicates whether its control or trreatment (trt)
# Metadata_target = show which gene is targeted with the given pertrubation
df["Metadata_target"].unique()

array([nan, 'CHRM1|CHRM2|CHRM3|CHRM4|CHRM5', 'HMGCR',
       'HDAC1|HDAC2|HDAC3|HDAC9', 'ERBB2', 'DNMT1|DNMT3A',
       'GABRA1|GABRA2|GABRA3|GABRA4|GABRA5|GABRA6', 'TUBB', 'KIF11',
       'PSMA1|PSMA2|PSMA3|PSMA4|PSMA5|PSMA6|PSMA7|PSMA8|PSMB1|PSMB10|PSMB11|PSMB2|PSMB3|PSMB4|PSMB5|PSMB6|PSMB7|PSMB8|PSMB9|PSMD1|PSMD2|RELA',
       'SQLE', 'GABRA1', 'KCNT2|TRPV4', 'AURKA|AURKB',
       'DRD2|GRIN2A|GRIN2B|GRIN2C|GRIN2D|GRIN3A', 'CFTR',
       'CACNA1C|CACNA1S|CACNA2D1|CACNG1|HTR3A|KCNA5',
       'ADRA1A|ADRA1B|ADRA2A|ADRA2B|ADRA2C|CHRM1|CHRM2|CHRM3|CHRM4|CHRM5|DRD1|DRD2|DRD3|DRD4|DRD5|HRH1|HTR1A|HTR1B|HTR1D|HTR1E|HTR2A|HTR2C|HTR3A|HTR6|HTR7',
       'EGFR|NR1I2', 'ADRA1A|ADRA2A|HRH1|HTR1A|HTR2A|HTR2B|HTR2C|SLC6A4',
       'EGFR|ERBB2', 'HIF1A', 'ESR1|ESR2|MAP1A|MAP2', 'SCN4A|SCN9A',
       'BIRC2|XIAP', 'AKT1|AKT2|AKT3|PRKG1', 'ACE',
       'HTR1A|HTR1B|HTR1D|HTR1E|HTR1F|HTR2A|HTR2B|HTR2C|HTR5A|HTR6|HTR7',
       'CYSLTR1|CYSLTR2', 'GAST', 'HTR1A', 'PSMB1', 'MET', 'NAE1|UBA3',
       'VD

In [4]:
df_activity = df.copy()

# make deafult value equal to row index
df_activity["Metadata_treatment_index"] = df_activity.index

# # make index equal to -1 for all DMSO treatment replicates
df_activity.loc[df["Metadata_broad_sample"] == "DMSO", "Metadata_treatment_index"] = -1

# # now all treatment replicates differ in the index column, except for DMSO replicates
df_activity.insert(
    0, "Metadata_treatment_index", df_activity.pop("Metadata_treatment_index")
)
df_activity

,Metadata_treatment_index,Metadata_broad_sample,Metadata_mg_per_ml,Metadata_mmoles_per_liter,Metadata_pert_id,Metadata_pert_mfc_id,Metadata_pert_well,Metadata_broad_sample_type,Metadata_pert_type,Metadata_broad_id,...,Nuclei_Texture_InverseDifferenceMoment_AGP_5_0,Nuclei_Texture_InverseDifferenceMoment_DNA_20_0,Nuclei_Texture_InverseDifferenceMoment_ER_5_0,Nuclei_Texture_InverseDifferenceMoment_Mito_10_0,Nuclei_Texture_InverseDifferenceMoment_Mito_5_0,Nuclei_Texture_SumAverage_RNA_5_0,Nuclei_Texture_SumEntropy_DNA_10_0,Nuclei_Texture_SumEntropy_DNA_20_0,Nuclei_Texture_SumEntropy_DNA_5_0,Nuclei_Texture_Variance_RNA_10_0
0,-1,DMSO,0.000000,0.000000,NaN,NaN,A01,control,control,NaN,...,-1.3544,-1.07770,2.26020,-0.377010,-0.065840,2.12360,2.8740,2.87500,2.3047,-0.92358
1,-1,DMSO,0.000000,0.000000,NaN,NaN,A02,control,control,NaN,...,-2.3840,-0.73440,1.12090,-0.182500,-0.061450,0.66985,2.3919,2.35230,1.8672,-0.11820
2,-1,DMSO,0.000000,0.000000,NaN,NaN,A03,control,control,NaN,...,-1.9493,-0.36148,0.44050,0.326660,0.547200,0.25015,1.2271,0.77847,1.0651,-0.44810
3,-1,DMSO,0.000000,0.000000,NaN,NaN,A04,control,control,NaN,...,-2.2909,-0.46380,0.96434,1.132200,0.753500,0.31403,1.4384,1.48110,1.2943,-0.83810
4,-1,DMSO,0.000000,0.000000,NaN,NaN,A05,control,control,NaN,...,-1.8955,-1.05350,1.64840,0.057781,0.070229,1.60990,1.1296,0.90213,1.1016,0.53225
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
379,379,BRD-K82746043-001-15-1,3.248700,3.333300,BRD-K82746043,BRD-K82746043-001-15-1,P20,trt,trt,BRD-K82746043,...,-6.1522,1.81410,1.54220,-1.874700,-1.133900,1.57540,-3.0962,-3.25160,-2.7683,1.40170
380,380,BRD-K82746043-001-15-1,1.082900,1.111100,BRD-K82746043,BRD-K82746043-001-15-1,P21,trt,trt,BRD-K82746043,...,-5.1586,1.50580,1.68420,-1.126400,-1.066600,1.24740,-1.5305,-1.79020,-1.2474,1.17600
381,381,BRD-K82746043-001-15-1,0.360970,0.370370,BRD-K82746043,BRD-K82746043-001-15-1,P22,trt,trt,BRD-K82746043,...,-5.9475,1.42100,1.51020,-1.103600,-1.666500,1.19840,-2.6086,-2.97620,-2.0026,0.91557
382,382,BRD-K82746043-001-15-1,0.120320,0.123460,BRD-K82746043,BRD-K82746043-001-15-1,P23,trt,trt,BRD-K82746043,...,-8.4408,2.99620,2.55230,-2.275200,-1.783500,2.49200,-4.3964,-4.19030,-3.8360,1.02240


Setting up parameters for average percision

In [5]:
# Metadata_broad_sample is a unique identifier 

# positive pairs are replicates of the same treament 
pos_sameby = ["Metadata_broad_sample"]
pos_diffby = []

neg_sameby = []

# this indicates pairs are replicates of different treaments
neg_diffby = ["Metadata_broad_sample", "Metadata_treatment_index"]


# next is to separate the metadata df and profiles df

# this is the metadata dataframe
meta = metadata = df_activity.filter(regex="^Metadata") 

# this is the morphlological profile dataframe
feats = profiles = df_activity.filter(regex="^(?!Metadata)").values



In [6]:
# now I am re-writing the average_percision function line by line  

# Combine all metadata columns needed for pair definitions
columns = flatten_str_list(pos_sameby, pos_diffby, neg_sameby, neg_diffby)
columns

['Metadata_broad_sample', 'Metadata_treatment_index']

In [7]:
# Validate and filter metadata to ensure the required columns are present
meta, columns = evaluate_and_filter(meta, columns)
validate_pipeline_input(meta, feats, columns)

In [8]:
meta

,Metadata_treatment_index,Metadata_broad_sample,Metadata_mg_per_ml,Metadata_mmoles_per_liter,Metadata_pert_id,Metadata_pert_mfc_id,Metadata_pert_well,Metadata_broad_sample_type,Metadata_pert_type,Metadata_broad_id,Metadata_InChIKey14,Metadata_moa,Metadata_target,Metadata_broad_date,Metadata_Well
0,-1,DMSO,0.000000,0.000000,NaN,NaN,A01,control,control,NaN,NaN,NaN,NaN,NaN,A01
1,-1,DMSO,0.000000,0.000000,NaN,NaN,A02,control,control,NaN,NaN,NaN,NaN,NaN,A02
2,-1,DMSO,0.000000,0.000000,NaN,NaN,A03,control,control,NaN,NaN,NaN,NaN,NaN,A03
3,-1,DMSO,0.000000,0.000000,NaN,NaN,A04,control,control,NaN,NaN,NaN,NaN,NaN,A04
4,-1,DMSO,0.000000,0.000000,NaN,NaN,A05,control,control,NaN,NaN,NaN,NaN,NaN,A05
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
379,379,BRD-K82746043-001-15-1,3.248700,3.333300,BRD-K82746043,BRD-K82746043-001-15-1,P20,trt,trt,BRD-K82746043,JLYAXFNOILIKPP,BCL inhibitor,BCL2|BCL2L1|BCL2L2,broad_id_20170327,P20
380,380,BRD-K82746043-001-15-1,1.082900,1.111100,BRD-K82746043,BRD-K82746043-001-15-1,P21,trt,trt,BRD-K82746043,JLYAXFNOILIKPP,BCL inhibitor,BCL2|BCL2L1|BCL2L2,broad_id_20170327,P21
381,381,BRD-K82746043-001-15-1,0.360970,0.370370,BRD-K82746043,BRD-K82746043-001-15-1,P22,trt,trt,BRD-K82746043,JLYAXFNOILIKPP,BCL inhibitor,BCL2|BCL2L1|BCL2L2,broad_id_20170327,P22
382,382,BRD-K82746043-001-15-1,0.120320,0.123460,BRD-K82746043,BRD-K82746043-001-15-1,P23,trt,trt,BRD-K82746043,JLYAXFNOILIKPP,BCL inhibitor,BCL2|BCL2L1|BCL2L2,broad_id_20170327,P23


In [9]:
# Get the distance function for similarity calculations (e.g., cosine)
distance = "cosine"
distance_fn = compute.get_distance_fn(distance)

# returns a callable function in this case it would be the cosine similarity function
distance_fn

<function copairs.compute.batch_processing.<locals>.batched_fn(feats: numpy.ndarray, pair_ix: numpy.ndarray, batch_size: int)>

In [10]:
# Reset metadata index for consistent indexing
meta = meta.reset_index(drop=True).copy()

meta

,Metadata_treatment_index,Metadata_broad_sample,Metadata_mg_per_ml,Metadata_mmoles_per_liter,Metadata_pert_id,Metadata_pert_mfc_id,Metadata_pert_well,Metadata_broad_sample_type,Metadata_pert_type,Metadata_broad_id,Metadata_InChIKey14,Metadata_moa,Metadata_target,Metadata_broad_date,Metadata_Well
0,-1,DMSO,0.000000,0.000000,NaN,NaN,A01,control,control,NaN,NaN,NaN,NaN,NaN,A01
1,-1,DMSO,0.000000,0.000000,NaN,NaN,A02,control,control,NaN,NaN,NaN,NaN,NaN,A02
2,-1,DMSO,0.000000,0.000000,NaN,NaN,A03,control,control,NaN,NaN,NaN,NaN,NaN,A03
3,-1,DMSO,0.000000,0.000000,NaN,NaN,A04,control,control,NaN,NaN,NaN,NaN,NaN,A04
4,-1,DMSO,0.000000,0.000000,NaN,NaN,A05,control,control,NaN,NaN,NaN,NaN,NaN,A05
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
379,379,BRD-K82746043-001-15-1,3.248700,3.333300,BRD-K82746043,BRD-K82746043-001-15-1,P20,trt,trt,BRD-K82746043,JLYAXFNOILIKPP,BCL inhibitor,BCL2|BCL2L1|BCL2L2,broad_id_20170327,P20
380,380,BRD-K82746043-001-15-1,1.082900,1.111100,BRD-K82746043,BRD-K82746043-001-15-1,P21,trt,trt,BRD-K82746043,JLYAXFNOILIKPP,BCL inhibitor,BCL2|BCL2L1|BCL2L2,broad_id_20170327,P21
381,381,BRD-K82746043-001-15-1,0.360970,0.370370,BRD-K82746043,BRD-K82746043-001-15-1,P22,trt,trt,BRD-K82746043,JLYAXFNOILIKPP,BCL inhibitor,BCL2|BCL2L1|BCL2L2,broad_id_20170327,P22
382,382,BRD-K82746043-001-15-1,0.120320,0.123460,BRD-K82746043,BRD-K82746043-001-15-1,P23,trt,trt,BRD-K82746043,JLYAXFNOILIKPP,BCL inhibitor,BCL2|BCL2L1|BCL2L2,broad_id_20170327,P23


In [11]:
 # Initialize the Matcher object to find pairs based on metadata rules

matcher = Matcher(meta, columns, seed=0)
matcher

In [12]:
# This identifies all the positive pairs 
pos_pairs = matcher.get_all_pairs(sameby=pos_sameby, diffby=pos_diffby)
pos_pairs

{'BRD-A69275535-001-01-5': [(np.int64(108), np.int64(112)),
  (np.int64(108), np.int64(113)),
  (np.int64(108), np.int64(109)),
  (np.int64(108), np.int64(110)),
  (np.int64(108), np.int64(111)),
  (np.int64(109), np.int64(112)),
  (np.int64(109), np.int64(113)),
  (np.int64(109), np.int64(110)),
  (np.int64(109), np.int64(111)),
  (np.int64(110), np.int64(112)),
  (np.int64(110), np.int64(113)),
  (np.int64(110), np.int64(111)),
  (np.int64(111), np.int64(112)),
  (np.int64(111), np.int64(113)),
  (np.int64(112), np.int64(113))],
 'BRD-A69636825-003-04-7': [(np.int64(132), np.int64(133)),
  (np.int64(132), np.int64(134)),
  (np.int64(132), np.int64(135)),
  (np.int64(132), np.int64(136)),
  (np.int64(132), np.int64(137)),
  (np.int64(133), np.int64(134)),
  (np.int64(133), np.int64(135)),
  (np.int64(133), np.int64(136)),
  (np.int64(133), np.int64(137)),
  (np.int64(134), np.int64(135)),
  (np.int64(134), np.int64(136)),
  (np.int64(134), np.int64(137)),
  (np.int64(135), np.int64(13

In [13]:
# then checks if the number of positive pairs == 0 then raise an error 
pos_total = sum(len(p) for p in pos_pairs.values())
if pos_total == 0:
    raise UnpairedException("Unable to find positive pairs.")

In [14]:
# Convert positive pairs to a NumPy array for efficient computation
pos_pairs = np.fromiter(
    itertools.chain.from_iterable(pos_pairs.values()),
    dtype=np.dtype((np.int32, 2)),
    count=pos_total,
)
pos_pairs

array([[108, 112],
       [108, 113],
       [108, 109],
       ...,
       [233, 126],
       [233, 127],
       [126, 127]], shape=(1248, 2), dtype=int32)

In [15]:
# Identify negative pairs based on `neg_sameby` and `neg_diffby`
neg_pairs = matcher.get_all_pairs(sameby=neg_sameby, diffby=neg_diffby)
neg_pairs

{None: [(np.int64(0), np.int64(6)),
  (np.int64(0), np.int64(7)),
  (np.int64(0), np.int64(8)),
  (np.int64(0), np.int64(9)),
  (np.int64(0), np.int64(10)),
  (np.int64(0), np.int64(11)),
  (np.int64(0), np.int64(12)),
  (np.int64(0), np.int64(13)),
  (np.int64(0), np.int64(14)),
  (np.int64(0), np.int64(15)),
  (np.int64(0), np.int64(16)),
  (np.int64(0), np.int64(17)),
  (np.int64(0), np.int64(18)),
  (np.int64(0), np.int64(19)),
  (np.int64(0), np.int64(20)),
  (np.int64(0), np.int64(21)),
  (np.int64(0), np.int64(22)),
  (np.int64(0), np.int64(23)),
  (np.int64(0), np.int64(30)),
  (np.int64(0), np.int64(31)),
  (np.int64(0), np.int64(32)),
  (np.int64(0), np.int64(33)),
  (np.int64(0), np.int64(34)),
  (np.int64(0), np.int64(35)),
  (np.int64(0), np.int64(36)),
  (np.int64(0), np.int64(37)),
  (np.int64(0), np.int64(38)),
  (np.int64(0), np.int64(39)),
  (np.int64(0), np.int64(40)),
  (np.int64(0), np.int64(41)),
  (np.int64(0), np.int64(42)),
  (np.int64(0), np.int64(43)),
  (np.

In [16]:
# check if there are negative pairs, if 0 raise an error
neg_total = sum(len(p) for p in neg_pairs.values())
if neg_total == 0:
    raise UnpairedException("Unable to find negative pairs.")

In [17]:
# Convert negative pairs to a NumPy array for efficient computation
neg_pairs = np.fromiter(
    itertools.chain.from_iterable(neg_pairs.values()),
    dtype=np.dtype((np.int32, 2)),
    count=neg_total,
)

In [18]:
# provides the row index to dictate the negative pairs 
neg_pairs

array([[  0,   6],
       [  0,   7],
       [  0,   8],
       ...,
       [377, 381],
       [377, 382],
       [377, 383]], shape=(72288, 2), dtype=int32)

In [19]:
# Compute similarities for positive pairs
batch_size = 20000

# here we are using the cosine similarity callable to calculate the 
# distance similarity between positive pairs
pos_sims = distance_fn(feats, pos_pairs, batch_size)

# these are all the calculated distances 
pos_sims


array([0.35714573, 0.14005443, 0.39402753, ..., 0.07142546, 0.11407731,
       0.71208286], shape=(1248,), dtype=float32)

In [20]:
# Compute similarities for negative pairs
neg_sims = distance_fn(feats, neg_pairs, batch_size)

neg_sims

array([ 0.12619643,  0.37474453,  0.42295352, ..., -0.02606241,
       -0.06152191, -0.04998211], shape=(72288,), dtype=float32)

In [21]:
# once all the distances are generated, we build a rank list 
paired_ix, rel_k_list, counts = build_rank_lists(
    pos_pairs, neg_pairs, pos_sims, neg_sims
)

In [22]:
# unique indices of profiles that appear in the rank list
paired_ix


array([  0,   1,   2,   3,   4,   5,   6,   7,   8,   9,  10,  11,  12,
        13,  14,  15,  16,  17,  18,  19,  20,  21,  22,  23,  24,  25,
        26,  27,  28,  29,  30,  31,  32,  33,  34,  35,  36,  37,  38,
        39,  40,  41,  42,  43,  44,  45,  46,  47,  48,  49,  50,  51,
        52,  53,  54,  55,  56,  57,  58,  59,  60,  61,  62,  63,  64,
        65,  66,  67,  68,  69,  70,  71,  72,  73,  74,  75,  76,  77,
        78,  79,  80,  81,  82,  83,  84,  85,  86,  87,  88,  89,  90,
        91,  92,  93,  94,  95,  96,  97,  98,  99, 100, 101, 102, 103,
       104, 105, 106, 107, 108, 109, 110, 111, 112, 113, 114, 115, 116,
       117, 118, 119, 120, 121, 122, 123, 124, 125, 126, 127, 128, 129,
       130, 131, 132, 133, 134, 135, 136, 137, 138, 139, 140, 141, 142,
       143, 144, 145, 146, 147, 148, 149, 150, 151, 152, 153, 154, 155,
       156, 157, 158, 159, 160, 161, 162, 163, 164, 165, 166, 167, 168,
       169, 170, 171, 172, 173, 174, 175, 176, 177, 178, 179, 18

In [23]:
# Array of relevance labels (1 for positive pairs, 0 for negative pairs), 
# sorted by similarity in descending order.
rel_k_list


array([1, 1, 0, ..., 0, 0, 0], shape=(147072,), dtype=int32)

In [24]:
#Array of counts indicating how many times each profile index appears in the rank lists3
counts


array([383, 383, 383, 383, 383, 383, 383, 383, 383, 383, 383, 383, 383,
       383, 383, 383, 383, 383, 383, 383, 383, 383, 383, 383, 383, 383,
       383, 383, 383, 383, 383, 383, 383, 383, 383, 383, 383, 383, 383,
       383, 383, 383, 383, 383, 383, 383, 383, 383, 383, 383, 383, 383,
       383, 383, 383, 383, 383, 383, 383, 383, 383, 383, 383, 383, 383,
       383, 383, 383, 383, 383, 383, 383, 383, 383, 383, 383, 383, 383,
       383, 383, 383, 383, 383, 383, 383, 383, 383, 383, 383, 383, 383,
       383, 383, 383, 383, 383, 383, 383, 383, 383, 383, 383, 383, 383,
       383, 383, 383, 383, 383, 383, 383, 383, 383, 383, 383, 383, 383,
       383, 383, 383, 383, 383, 383, 383, 383, 383, 383, 383, 383, 383,
       383, 383, 383, 383, 383, 383, 383, 383, 383, 383, 383, 383, 383,
       383, 383, 383, 383, 383, 383, 383, 383, 383, 383, 383, 383, 383,
       383, 383, 383, 383, 383, 383, 383, 383, 383, 383, 383, 383, 383,
       383, 383, 383, 383, 383, 383, 383, 383, 383, 383, 383, 38

In [ ]:
# next s
ap_scores, null_confs = compute.ap_contiguous(rel_k_list, counts)


In [28]:
ap_scores

array([0.28889179, 0.2319961 , 0.44693104, 0.31095992, 0.14105653,
       0.13326486, 0.05092226, 0.30890383, 0.41251337, 0.37773026,
       0.7155914 , 0.58591479, 0.0382547 , 0.27625397, 0.64176471,
       0.25465244, 0.18464803, 0.1662895 , 0.13062917, 0.46833551,
       0.81882353, 0.59458653, 0.67143945, 0.37272063, 0.45643524,
       0.40924851, 0.36069853, 0.33633516, 0.21647636, 0.18176443,
       0.6968254 , 0.15523081, 0.63030303, 0.43621934, 0.55413534,
       0.53088426, 0.05211318, 0.23937841, 0.31397993, 0.10578179,
       0.17372453, 0.15250299, 0.85263158, 0.89090909, 1.        ,
       0.96666667, 0.9       , 0.58653846, 0.28077381, 0.27125448,
       0.23988125, 0.10666844, 0.36926537, 0.34078879, 0.85285714,
       0.77833333, 1.        , 1.        , 1.        , 1.        ,
       0.08086501, 0.83225806, 0.83030303, 0.81666667, 0.82272727,
       0.82325581, 0.94194041, 1.        , 1.        , 1.        ,
       1.        , 1.        , 0.09856895, 0.20381795, 0.23524

In [ ]:
# next step is to attach the average precision score into the metadata df:
meta["n_pos_pairs"] = 0
meta["n_total_pairs"] = 0
meta.loc[paired_ix, "average_precision"] = ap_scores
meta.loc[paired_ix, "n_pos_pairs"] = null_confs[:, 0]
meta.loc[paired_ix, "n_total_pairs"] = null_confs[:, 1]

In [35]:
replicate_aps = meta
replicate_aps

,Metadata_treatment_index,Metadata_broad_sample,Metadata_mg_per_ml,Metadata_mmoles_per_liter,Metadata_pert_id,Metadata_pert_mfc_id,Metadata_pert_well,Metadata_broad_sample_type,Metadata_pert_type,Metadata_broad_id,Metadata_InChIKey14,Metadata_moa,Metadata_target,Metadata_broad_date,Metadata_Well,n_pos_pairs,n_total_pairs,average_precision
0,-1,DMSO,0.000000,0.000000,NaN,NaN,A01,control,control,NaN,NaN,NaN,NaN,NaN,A01,23,383,0.288892
1,-1,DMSO,0.000000,0.000000,NaN,NaN,A02,control,control,NaN,NaN,NaN,NaN,NaN,A02,23,383,0.231996
2,-1,DMSO,0.000000,0.000000,NaN,NaN,A03,control,control,NaN,NaN,NaN,NaN,NaN,A03,23,383,0.446931
3,-1,DMSO,0.000000,0.000000,NaN,NaN,A04,control,control,NaN,NaN,NaN,NaN,NaN,A04,23,383,0.310960
4,-1,DMSO,0.000000,0.000000,NaN,NaN,A05,control,control,NaN,NaN,NaN,NaN,NaN,A05,23,383,0.141057
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
379,379,BRD-K82746043-001-15-1,3.248700,3.333300,BRD-K82746043,BRD-K82746043-001-15-1,P20,trt,trt,BRD-K82746043,JLYAXFNOILIKPP,BCL inhibitor,BCL2|BCL2L1|BCL2L2,broad_id_20170327,P20,5,383,0.726786
380,380,BRD-K82746043-001-15-1,1.082900,1.111100,BRD-K82746043,BRD-K82746043-001-15-1,P21,trt,trt,BRD-K82746043,JLYAXFNOILIKPP,BCL inhibitor,BCL2|BCL2L1|BCL2L2,broad_id_20170327,P21,5,383,0.658824
381,381,BRD-K82746043-001-15-1,0.360970,0.370370,BRD-K82746043,BRD-K82746043-001-15-1,P22,trt,trt,BRD-K82746043,JLYAXFNOILIKPP,BCL inhibitor,BCL2|BCL2L1|BCL2L2,broad_id_20170327,P22,5,383,0.517619
382,382,BRD-K82746043-001-15-1,0.120320,0.123460,BRD-K82746043,BRD-K82746043-001-15-1,P23,trt,trt,BRD-K82746043,JLYAXFNOILIKPP,BCL inhibitor,BCL2|BCL2L1|BCL2L2,broad_id_20170327,P23,5,383,0.543290


In [34]:
# additional step: (shown in the tutorial) -> remove the DMSO scores
replicate_aps = replicate_aps.query("Metadata_broad_sample != 'DMSO'")  # remove DMSO
replicate_aps

,Metadata_treatment_index,Metadata_broad_sample,Metadata_mg_per_ml,Metadata_mmoles_per_liter,Metadata_pert_id,Metadata_pert_mfc_id,Metadata_pert_well,Metadata_broad_sample_type,Metadata_pert_type,Metadata_broad_id,Metadata_InChIKey14,Metadata_moa,Metadata_target,Metadata_broad_date,Metadata_Well,n_pos_pairs,n_total_pairs,average_precision
6,6,BRD-K74363950-004-01-0,5.655600,10.000000,BRD-K74363950,BRD-K74363950-004-01-0,A07,trt,trt,BRD-K74363950,ASMXXROZKSBQIH,acetylcholine receptor antagonist,CHRM1|CHRM2|CHRM3|CHRM4|CHRM5,broad_id_20170327,A07,5,383,0.050922
7,7,BRD-K74363950-004-01-0,1.885200,3.333300,BRD-K74363950,BRD-K74363950-004-01-0,A08,trt,trt,BRD-K74363950,ASMXXROZKSBQIH,acetylcholine receptor antagonist,CHRM1|CHRM2|CHRM3|CHRM4|CHRM5,broad_id_20170327,A08,5,383,0.308904
8,8,BRD-K74363950-004-01-0,0.628400,1.111100,BRD-K74363950,BRD-K74363950-004-01-0,A09,trt,trt,BRD-K74363950,ASMXXROZKSBQIH,acetylcholine receptor antagonist,CHRM1|CHRM2|CHRM3|CHRM4|CHRM5,broad_id_20170327,A09,5,383,0.412513
9,9,BRD-K74363950-004-01-0,0.209470,0.370370,BRD-K74363950,BRD-K74363950-004-01-0,A10,trt,trt,BRD-K74363950,ASMXXROZKSBQIH,acetylcholine receptor antagonist,CHRM1|CHRM2|CHRM3|CHRM4|CHRM5,broad_id_20170327,A10,5,383,0.377730
10,10,BRD-K74363950-004-01-0,0.069823,0.123460,BRD-K74363950,BRD-K74363950-004-01-0,A11,trt,trt,BRD-K74363950,ASMXXROZKSBQIH,acetylcholine receptor antagonist,CHRM1|CHRM2|CHRM3|CHRM4|CHRM5,broad_id_20170327,A11,5,383,0.715591
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
379,379,BRD-K82746043-001-15-1,3.248700,3.333300,BRD-K82746043,BRD-K82746043-001-15-1,P20,trt,trt,BRD-K82746043,JLYAXFNOILIKPP,BCL inhibitor,BCL2|BCL2L1|BCL2L2,broad_id_20170327,P20,5,383,0.726786
380,380,BRD-K82746043-001-15-1,1.082900,1.111100,BRD-K82746043,BRD-K82746043-001-15-1,P21,trt,trt,BRD-K82746043,JLYAXFNOILIKPP,BCL inhibitor,BCL2|BCL2L1|BCL2L2,broad_id_20170327,P21,5,383,0.658824
381,381,BRD-K82746043-001-15-1,0.360970,0.370370,BRD-K82746043,BRD-K82746043-001-15-1,P22,trt,trt,BRD-K82746043,JLYAXFNOILIKPP,BCL inhibitor,BCL2|BCL2L1|BCL2L2,broad_id_20170327,P22,5,383,0.517619
382,382,BRD-K82746043-001-15-1,0.120320,0.123460,BRD-K82746043,BRD-K82746043-001-15-1,P23,trt,trt,BRD-K82746043,JLYAXFNOILIKPP,BCL inhibitor,BCL2|BCL2L1|BCL2L2,broad_id_20170327,P23,5,383,0.543290


In [37]:
# now that we have all the average position scores, we take the mean in order to get the mAP score value 
ap_scores = replicate_aps

# Filter out invalid or incomplete AP scores
ap_scores = ap_scores.query("~average_precision.isna() and n_pos_pairs > 0")
ap_scores = ap_scores.reset_index(drop=True).copy()


ap_scores

,Metadata_treatment_index,Metadata_broad_sample,Metadata_mg_per_ml,Metadata_mmoles_per_liter,Metadata_pert_id,Metadata_pert_mfc_id,Metadata_pert_well,Metadata_broad_sample_type,Metadata_pert_type,Metadata_broad_id,Metadata_InChIKey14,Metadata_moa,Metadata_target,Metadata_broad_date,Metadata_Well,n_pos_pairs,n_total_pairs,average_precision
0,-1,DMSO,0.000000,0.000000,NaN,NaN,A01,control,control,NaN,NaN,NaN,NaN,NaN,A01,23,383,0.288892
1,-1,DMSO,0.000000,0.000000,NaN,NaN,A02,control,control,NaN,NaN,NaN,NaN,NaN,A02,23,383,0.231996
2,-1,DMSO,0.000000,0.000000,NaN,NaN,A03,control,control,NaN,NaN,NaN,NaN,NaN,A03,23,383,0.446931
3,-1,DMSO,0.000000,0.000000,NaN,NaN,A04,control,control,NaN,NaN,NaN,NaN,NaN,A04,23,383,0.310960
4,-1,DMSO,0.000000,0.000000,NaN,NaN,A05,control,control,NaN,NaN,NaN,NaN,NaN,A05,23,383,0.141057
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
379,379,BRD-K82746043-001-15-1,3.248700,3.333300,BRD-K82746043,BRD-K82746043-001-15-1,P20,trt,trt,BRD-K82746043,JLYAXFNOILIKPP,BCL inhibitor,BCL2|BCL2L1|BCL2L2,broad_id_20170327,P20,5,383,0.726786
380,380,BRD-K82746043-001-15-1,1.082900,1.111100,BRD-K82746043,BRD-K82746043-001-15-1,P21,trt,trt,BRD-K82746043,JLYAXFNOILIKPP,BCL inhibitor,BCL2|BCL2L1|BCL2L2,broad_id_20170327,P21,5,383,0.658824
381,381,BRD-K82746043-001-15-1,0.360970,0.370370,BRD-K82746043,BRD-K82746043-001-15-1,P22,trt,trt,BRD-K82746043,JLYAXFNOILIKPP,BCL inhibitor,BCL2|BCL2L1|BCL2L2,broad_id_20170327,P22,5,383,0.517619
382,382,BRD-K82746043-001-15-1,0.120320,0.123460,BRD-K82746043,BRD-K82746043-001-15-1,P23,trt,trt,BRD-K82746043,JLYAXFNOILIKPP,BCL inhibitor,BCL2|BCL2L1|BCL2L2,broad_id_20170327,P23,5,383,0.543290


In [ ]:
# Extract configurations for null distribution generation
# identifying and grouping the different combinations of conditions (or configurations) in the data that will be used to create the null distributions.
null_confs = ap_scores[["n_pos_pairs", "n_total_pairs"]].values
null_confs, rev_ix = np.unique(null_confs, axis=0, return_inverse=True)

In [39]:
null_confs

array([[  5, 383],
       [ 11, 383],
       [ 23, 383]])

In [40]:
rev_ix


array([2, 2, 2, 2, 2, 2, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 2, 2, 2, 2, 2, 2, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       1, 1, 1, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 2, 2, 2, 2, 2, 2,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 1, 1, 1, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 2, 2, 2, 2, 2, 2, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 1, 1, 1, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,

In [44]:
# Generate null distributions for each unique configuration
null_size=10000
seed=0
null_dists = compute.get_null_dists(null_confs, null_size, seed=seed)
ap_scores["null_ix"] = rev_ix
null_dists

array([[0.11827002, 0.08022756, 0.11339746, ..., 0.01138769, 0.01706163,
        0.0107111 ],
       [0.0374098 , 0.03500608, 0.03720492, ..., 0.0429553 , 0.02890686,
        0.0403944 ],
       [0.06712543, 0.05824288, 0.06095092, ..., 0.0598328 , 0.06662653,
        0.05400726]], shape=(3, 10000), dtype=float32)

In [46]:
def get_p_value(params):
    map_score, indices = params
    null_dist = null_dists[rev_ix[indices]].mean(axis=0)
    num = (null_dist > map_score).sum()
    p_value = (num + 1) / (null_size + 1)  # Add 1 for stability
    return p_value


In [48]:
# Group by the specified metadata column(s) and calculate mean AP
sameby = pos_sameby
map_scores = ap_scores.groupby(sameby, observed=True).agg(
    {
        "average_precision": ["mean", lambda x: list(x.index)],
    }
)
map_scores.columns = ["mean_average_precision", "indices"]
map_scores

,mean_average_precision,indices
Metadata_broad_sample,,
BRD-A69275535-001-01-5,0.203576,"[108, 109, 110, 111, 112, 113]"
BRD-A69636825-003-04-7,0.269093,"[132, 133, 134, 135, 136, 137]"
BRD-A69815203-001-07-6,0.862226,"[312, 313, 314, 315, 316, 317]"
BRD-A70858459-001-01-7,0.351816,"[174, 175, 176, 177, 178, 179]"
BRD-A72309220-001-04-1,0.263986,"[210, 211, 212, 213, 214, 215]"
BRD-A72390365-001-15-2,0.554667,"[234, 235, 236, 237, 238, 239]"
BRD-A73368467-003-17-6,0.788666,"[270, 271, 272, 273, 274, 275]"
BRD-A74980173-001-11-9,0.500600,"[30, 31, 32, 33, 34, 35]"
BRD-A81233518-004-16-1,0.140208,"[330, 331, 332, 333, 334, 335]"


In [50]:
# Compute p-values for each group using the null distributions
params = map_scores[["mean_average_precision", "indices"]]
map_scores["p_value"] = thread_map(get_p_value, params.values, leave=False)

In [52]:
# Perform multiple testing correction on p-values
reject, pvals_corrected, alphacSidak, alphacBonf = multipletests(
    map_scores["p_value"], method="fdr_bh"
)
map_scores["corrected_p_value"] = pvals_corrected

In [53]:
# Mark scores below the p-value threshold
threshold = 0.05
map_scores["below_p"] = map_scores["p_value"] < threshold
map_scores["below_corrected_p"] = map_scores["corrected_p_value"] < threshold

In [54]:
# Mark scores below the p-value threshold
map_scores["below_p"] = map_scores["p_value"] < threshold
map_scores["below_corrected_p"] = map_scores["corrected_p_value"] < threshold

In [55]:
map_scores

,mean_average_precision,indices,p_value,corrected_p_value,below_p,below_corrected_p
Metadata_broad_sample,,,,,,
BRD-A69275535-001-01-5,0.203576,"[108, 109, 110, 111, 112, 113]",0.012899,0.016318,True,True
BRD-A69636825-003-04-7,0.269093,"[132, 133, 134, 135, 136, 137]",0.000800,0.001348,True,True
BRD-A69815203-001-07-6,0.862226,"[312, 313, 314, 315, 316, 317]",0.000100,0.000281,True,True
BRD-A70858459-001-01-7,0.351816,"[174, 175, 176, 177, 178, 179]",0.000200,0.000393,True,True
BRD-A72309220-001-04-1,0.263986,"[210, 211, 212, 213, 214, 215]",0.000900,0.001475,True,True
BRD-A72390365-001-15-2,0.554667,"[234, 235, 236, 237, 238, 239]",0.000100,0.000281,True,True
BRD-A73368467-003-17-6,0.788666,"[270, 271, 272, 273, 274, 275]",0.000100,0.000281,True,True
BRD-A74980173-001-11-9,0.500600,"[30, 31, 32, 33, 34, 35]",0.000100,0.000281,True,True
BRD-A81233518-004-16-1,0.140208,"[330, 331, 332, 333, 334, 335]",0.015598,0.018642,True,True
